<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Using Sessions with LlamaIndex</h1>

A Session is a sequence of traces representing a user's interaction with an application.

In this tutorial, you will:
- Build and trace a simple LlamaIndex application
- Use sessions to organize traces

ℹ️ This notebook requires an OpenAI API key.

## 1. Install Dependencies and Import Libraries

In [ ]:
%pip install -Uq "arize-phoenix[llama-index]" openinference-instrumentation-llama-index gcsfs faker

In [ ]:
import json
import os
from getpass import getpass
from random import sample
from urllib.request import urlopen
from uuid import uuid4

from faker import Faker
from gcsfs import GCSFileSystem
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from openinference.instrumentation import using_session, using_user
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from tqdm import tqdm

import phoenix as px
from phoenix.otel import register

## 2. Configure Your OpenAI API Key

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

## 3. Configure the default project then Launch Phoenix



🚨 Phoenix is configured with environment variables. 🚨

In this tutorial we want to change the default project we send traces to by modifying the `PHOENIX_PROJECT_NAME` environment variable defined blow.

In [ ]:
os.environ["PHOENIX_PROJECT_NAME"] = "SESSIONS-DEMO"

Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [ ]:
tracer_provider = register(endpoint="http://127.0.0.1:6006/v1/traces")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

Launch Phoenix

In [ ]:
px.launch_app()

## 4. Build Your LlamaIndex Application

This example uses a `RetrieverQueryEngine` over a pre-built index of the Arize documentation, but you can use whatever LlamaIndex application you like.

Download our pre-built index of the Arize docs from cloud storage and instantiate your storage context.

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
persist_dir = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(fs=file_system, persist_dir=persist_dir)

We are now ready to instantiate our query engine that will perform retrieval-augmented generation (RAG). Query engine is a generic interface in LlamaIndex that allows you to ask question over your data. A query engine takes in a natural language query, and returns a rich response. It is built on top of Retrievers. You can compose multiple query engines to achieve more advanced capability

In [ ]:
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding()
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

# 5. Download Sample Queries

In [ ]:
queries_url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
with urlopen(queries_url) as response:
    queries = [json.loads(line)["query"] for line in response]

# 6. Group Queries By User Sessions

In [ ]:
session_id = str(uuid4())
session_user = Faker().user_name()

with using_session(session_id), using_user(session_user):
    for query in tqdm(sample(queries, 3)):
        query_engine.query(query)

<video controls src="https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/llama-index-knowledge-base-tutorial/project_sessions.mov" />